##### we've already generated 100 scenarios in an Excel File
##### from now on we follow the steps of scenario reduction algorithm

In [26]:
# IMPORTING REQUIRED LIBRARIES
import pandas as pd
import numpy as np
from math import  inf
import matplotlib.pyplot as plt

# FIRST WE IMPORT OUR SCENARIOS INTO A VARIABLE
scenarios=pd.read_excel('scenarios.xlsx',header=None)
# LET'S TAKE A LOOK
scenarios.head(10)
# FIRST THREE COLUMN OF OUR DATASET CONSIST OF OUR SCENARIOS
# SO WE NEED TO FILTER IT OUT FIRST AND PUT IT A NEW VARIABLE WE LIKE TO CALL MAIN_SCENARIOS
main_scenarios=scenarios.iloc[:,:3]
# WE ALSO NEED Probabilities OF EACH SCENARIO
probabilities=np.array(scenarios.iloc[:,5])
# NOW WE DEFINE OUR DISTANCE MATRIX
scenario_distance_matrix=np.zeros([100,100])

#FIRST WE DEFINE OUR minimum AND epsilon
minimum=0
epsilon=0.065

# (minimum < epsilon) makes sure that the min distance between scenarios does not surpass epsilon

while minimum<epsilon:
    n=scenario_distance_matrix.shape[0]      #this gives out number of rows in our distance matrix
    m=scenario_distance_matrix.shape[1]      #this gives out number of columns in our distance matrix
    for i in range(n):   
        for j in range(m): 
            if i==j:
                scenario_distance_matrix[i][j]=inf   # we set the distance of each scenario from itself to infinite
            else:
                a=np.array(main_scenarios.iloc[i])   # take scenario a
                b=np.array(main_scenarios.iloc[j])   # take scenario b
                dist=np.sqrt(np.sum((a-b)**2, axis=0))  # calculate the distance between these two
                scenario_distance_matrix[i][j]=dist*probabilities[i]  # update distance matrix
    minimum=scenario_distance_matrix.min()

    # NOW WE SEARCH TO SEE WHICH ROW AND COLUMN CONTAINS THE MINIMUMM VALUE
    # THEN WE PROCEED TO DELETE IT

    for i in range(n):
        for j in range(m):
            if scenario_distance_matrix[i][j]==minimum:
                row=i
                column=j
                break   

    #update scenarios
    # we delete the row and column which contains the min value from distance matrix 
    minimum=scenario_distance_matrix[row,column]
    probabilities[column]=probabilities[row]+probabilities[column]   # we add the deleted scenario probabilty to the one we kept
    probabilities=np.delete(probabilities,row,axis=0)                # we update our probability array for next loop
    # NOW WE UPDATE THE MAIN SCENARIOS DATAFRAME AND ALSO THE DISTANCE MATRIX
    main_scenarios=main_scenarios.drop(row).reset_index().drop(["index"], axis=1)
    scenario_distance_matrix=np.delete(scenario_distance_matrix,row,axis=0)
    scenario_distance_matrix=np.delete(scenario_distance_matrix,column,axis=1)
    # This Line of code is optional
    # here we wanted to make sure that our dataframe is reduced to 10 scenarion, no more or less
    if len(main_scenarios)==10:
        break


,0,1,2
0,1.495,1.794,11.96
1,1.765,2.118,14.12
2,2.035,2.442,16.28
3,2.305,2.766,18.44
4,2.515,3.018,20.12
5,2.635,3.162,21.08
6,2.755,3.306,22.04
7,2.875,3.450,23.00
8,3.115,3.738,24.92
9,3.415,4.098,27.32


In [29]:
#let's see the new scenarios
print('reduced scenarios \n',main_scenarios)
print('\nupdated probabilities \n',probabilities)  

reduced scenarios 
        0      1      2
0  1.495  1.794  11.96
1  1.765  2.118  14.12
2  2.035  2.442  16.28
3  2.305  2.766  18.44
4  2.515  3.018  20.12
5  2.635  3.162  21.08
6  2.755  3.306  22.04
7  2.875  3.450  23.00
8  3.115  3.738  24.92
9  3.415  4.098  27.32

updated probabilities 
 [0.03957961 0.07414016 0.13955724 0.19761466 0.1425933  0.08955143
 0.0798512  0.12069283 0.06859101 0.04512876]


### solving farmer problem with reduced scnearios

In [33]:
from docplex.mp.model import Model
FARMER=Model(name='farmer problem using reduced scenario')
#parameters
p=probabilities
wheat_yield = main_scenarios[0]
corn_yield = main_scenarios[1]
sugarbeet_yield = main_scenarios[2]
planting_cost=[150,230,260]
buying_price=[238,210]
selling_price=[170, 150, 36, 10]
#variables
x=FARMER.continuous_var_matrix(range(1,4),range(1,3),name='x',key_format='%s')
y=FARMER.continuous_var_cube(range(1,3),range(1,11),range(2,4),name='y',key_format='%s')
w=FARMER.continuous_var_cube(range(1,5),range(1,11),range(2,4),name='w',key_format='%s')
#constraints

# PLANTING CONSTRAINT FOR WHEAT

for t in range(1,3):
    FARMER.add_constraint_(x[1,t]+x[2,t]+x[3,t]<=500)

# LEAST AMOUNT REQUIRED FOR WHEAT CONSTRAINTS

for t in range(1,3):
    for s in range(1,11):
        FARMER.add_constraint_(wheat_yield[s-1]*x[1,t]+y[1,s,t+1]-w[1,s,t+1]>=200)

# LEAST AMOUNT REQUIRED FOR CORN CONSTRAINTS   
for t in range(1,3):
    for s in range(1,11):
        FARMER.add_constraint_(corn_yield[s-1]*x[2,t]+y[2,s,t+1]-w[2,s,t+1]>=240)

# SELLING AMOUNT SHOULD NOT EXCEED THE CULTIVATION         
for t in range(1,3):
    for s in range(1,11):
        FARMER.add_constraint_(sugarbeet_yield[s-1]*x[3,t]-w[3,s,t+1]-w[4,s,t+1]>=0)

# CONSTRAINT ON SELLING AMOUNT WITH HIGHEST PRICE FOR SUGARBEET
for t in range(2,4):
    for s in range(1,11):
        FARMER.add_constraint_(w[3,s,t]<=6000)

# CAN'T PLANT SUGARBEET ON THE SAME FARM FOR TWO YEARS STRAIGHT
FARMER.add_constraint_(x[3,2]<=x[1,1]+x[2,1])

# DEFINE OBJECTIVE FUNCTION
FARMER.minimize(sum(planting_cost[i]*x[i+1,t] for i in range(3) for t in range(1,3))+
sum(probabilities[s-1]*(buying_price[i-1]*y[i,s,t]-selling_price[k-1]*w[k,s,t]) for s in range(1,11)
for i in range(1,3) for t in range(2,4) for k in range(1,5)))

#PRINT solution
Solution=FARMER.solve()
Solution.display()




solution for: farmer problem using reduced scenario
objective: -627874.713
x11 = 133.779
x12 = 133.779
x21 = 113.314
x22 = 119.127
x31 = 252.906
x32 = 247.094
y212 = 36.714
y213 = 26.286
w122 = 36.120
w123 = 36.120
w132 = 72.241
w133 = 72.241
w142 = 108.361
w143 = 108.361
w152 = 136.455
w153 = 136.455
w162 = 152.508
w163 = 152.508
w172 = 168.562
w173 = 168.562
w182 = 184.615
w183 = 184.615
w192 = 216.722
w193 = 216.722
w1102 = 256.856
w1103 = 256.856
w223 = 12.311
w232 = 36.714
w233 = 50.908
w242 = 73.428
w243 = 89.505
w252 = 101.983
w253 = 119.525
w262 = 118.300
w263 = 136.680
w272 = 134.618
w273 = 153.834
w282 = 150.935
w283 = 170.988
w292 = 183.569
w293 = 205.297
w2102 = 224.363
w2103 = 248.183
w312 = 3024.759
w313 = 2955.241
w322 = 3571.037
w323 = 3488.963
w332 = 4117.314
w333 = 4022.686
w342 = 4663.592
w343 = 4556.408
w352 = 5088.475
w353 = 4971.525
w362 = 5331.265
w363 = 5208.735
w372 = 5574.055
w373 = 5445.945
w382 = 5816.845
w383 = 5683.155
w392 = 6000.000
w393 = 6000.000
w3102